In [1]:
%load_ext autoreload
%load_ext autotime

time: 342 µs (started: 2023-01-24 20:02:28 -08:00)


In [2]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 2.95 s (started: 2023-01-24 20:02:54 -08:00)


In [3]:
import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import json
import math
import numpy as np
import os
import pandas as pd
from dask import compute, delayed
from dask.dataframe import DataFrame
from dask.distributed import Client, LocalCluster, fire_and_forget, wait, worker_client
from wisio.utils.file_utils import ensure_dir
from wisio.utils.json_encoders import NpEncoder
from wisio.utils.logger import create_logger, format_log


time: 1.65 s (started: 2023-01-24 20:03:16 -08:00)


In [4]:
client = Client(LocalCluster(n_workers=32, local_directory=f"/var/tmp/playground-quartz/local"))
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 32
Total threads: 64,Total memory: 125.50 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43246,Workers: 32
Dashboard: http://127.0.0.1:8787/status,Total threads: 64
Started: Just now,Total memory: 125.50 GiB
Comm: tcp://127.0.0.1:37340,Total threads: 2
Dashboard: http://127.0.0.1:40355/status,Memory: 3.92 GiB
Nanny: tcp://127.0.0.1:43579,


time: 4.33 s (started: 2023-01-24 20:03:23 -08:00)


In [7]:
import json

def load_global_min_max(log_dir: str):
    with open(f"{log_dir}/global.json") as file:
        global_min_max = json.load(file)
    return global_min_max

time: 2.4 ms (started: 2023-01-24 20:04:55 -08:00)


In [41]:
DELTA = 0.0001


time: 708 µs (started: 2023-01-24 20:24:04 -08:00)


In [103]:
log_dir = "/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet"
ddf = dd.read_parquet(f"{log_dir}/*.parquet")
# ddf = dd.read_parquet(f"{log_dir}/*.parquet", engine='pyarrow', filters=[[('file_id', '=', 4531734247156679037)], [('file_id', '=', 4531734247515089012)]])
ddf

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
npartitions=1,,,,,,,,,,,,,,,,,,,,
,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,int64,int32,float32,float32,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 67.3 ms (started: 2023-01-24 21:17:24 -08:00)


In [104]:
tmid_min, tmid_max = load_global_min_max(log_dir=log_dir)['tmid']
trange = np.arange(tmid_min, tmid_max, 1e7)
# ddf = ddf.map_partitions(lambda df: df.assign(trange=pd.cut(df['tmid'], trange, right=True)))
ddf = ddf.map_partitions(lambda df: df.assign(trange=np.digitize(df['tmid'], trange, right=True)))

time: 17.2 ms (started: 2023-01-24 21:17:25 -08:00)


In [105]:
ddf['acc_pat'] = ddf['acc_pat'].astype('i1')
ddf['io_cat'] = ddf['io_cat'].astype('i1')

all_agg_persisted = ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration': [sum],
        'acc_pat': [min, max],
        'size': [min, max, sum],
        'index': ['count']
    }) \
    .reset_index() \
    .persist()


time: 103 ms (started: 2023-01-24 21:17:25 -08:00)


In [106]:
wait(all_agg_persisted)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('reset_index-e0146f02c9280b18611f9ac1161627d2', 0)>}, not_done=set())

time: 4.48 ms (started: 2023-01-24 21:17:25 -08:00)


In [107]:
trange_view_persisted = all_agg_persisted[['trange', 'duration']] \
    .groupby(['trange']) \
    .sum() \
    .sort_values(('duration', 'sum'), ascending=False) \
    .persist()


time: 32.2 ms (started: 2023-01-24 21:17:26 -08:00)


In [108]:
file_id_view_persisted = all_agg_persisted[['file_id', 'duration']] \
    .groupby(['file_id']) \
    .sum() \
    .sort_values(('duration', 'sum'), ascending=False) \
    .persist()


time: 32.9 ms (started: 2023-01-24 21:17:27 -08:00)


In [109]:
proc_id_view_persisted = all_agg_persisted[['proc_id', 'duration']] \
    .groupby(['proc_id']) \
    .sum() \
    .sort_values(('duration', 'sum'), ascending=False) \
    .persist()


time: 32.4 ms (started: 2023-01-24 21:17:27 -08:00)


In [110]:
dask.distributed.wait([trange_view_persisted, file_id_view_persisted, proc_id_view_persisted])


DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('sort_values-1cff2863e38b6ee6016f0f2499b2bf7a', 0)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('sort_values-83d309615f3c6641f4ad7661104553ad', 0)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('sort_values-04b8bb9fc7f87861f9d69bf284eb9545', 0)>}, not_done=set())

time: 17.6 ms (started: 2023-01-24 21:17:27 -08:00)


In [111]:
trange_view_persisted.head()


,duration
,sum
trange,
5,66.775764
450,5.333155
668,4.466336
344,4.180270
233,3.813165


time: 33.6 ms (started: 2023-01-24 21:17:28 -08:00)


In [112]:
all_agg_persisted._meta

Empty DataFrame
Columns: [(trange, ), (file_id, ), (proc_id, ), (io_cat, ), (duration, sum), (acc_pat, min), (acc_pat, max), (size, min), (size, max), (size, sum), (index, count)]
Index: []

time: 13.7 ms (started: 2023-01-24 21:17:29 -08:00)


In [113]:
all_agg_persisted.index

Dask Index Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: reset_index, 2 tasks

time: 8.64 ms (started: 2023-01-24 21:17:30 -08:00)


In [114]:
trange_view_persisted.index

Dask Index Structure:
npartitions=1
    int64
      ...
Name: trange, dtype: int64
Dask Name: sort_values, 2 tasks

time: 8.21 ms (started: 2023-01-24 21:17:31 -08:00)


In [115]:
trange_view_filtered = trange_view_persisted \
    .map_partitions(lambda df: df.assign(csp=df[('duration', 'sum')].cumsum() / df[('duration', 'sum')].sum())) \
    .map_partitions(lambda df: df.assign(delta=df['csp'].diff().fillna(df['csp'])))

trange_view_filtered = trange_view_filtered \
    .loc[trange_view_filtered['delta'] > DELTA] \
    .drop(columns=['csp', 'delta']) \
    .persist()

trange_view_filtered


,duration
,sum
npartitions=1,
,float32
,...


time: 60.1 ms (started: 2023-01-24 21:17:33 -08:00)


In [116]:
trange_view_filtered.head()

,duration
,sum
trange,
5,66.775764
450,5.333155
668,4.466336
344,4.180270
233,3.813165


time: 35 ms (started: 2023-01-24 21:17:34 -08:00)


In [125]:
trange_view_indices = list(set(trange_view_filtered.index.compute()))
trange_llc = all_agg_persisted[all_agg_persisted['trange'].isin(trange_view_indices)] \
    .groupby(['trange']) \
    .sum() \
    .sort_values(('duration', 'sum'), ascending=False) \
    .persist()

trange_llc


Dask DataFrame Structure:
              file_id proc_id io_cat duration acc_pat         size                index
                                          sum     min   max    min    max    sum  count
npartitions=1                                                                          
                int64   int64  int64  float32    int8  int8  int64  int64  int64  int64
                  ...     ...    ...      ...     ...   ...    ...    ...    ...    ...
Dask Name: sort_values, 1 tasks

time: 117 ms (started: 2023-01-24 21:51:08 -08:00)


In [126]:
wait(trange_llc)


DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('sort_values-153fa17be5e8a47c5fd77c83130dda79', 0)>}, not_done=set())

time: 6.19 ms (started: 2023-01-24 21:51:09 -08:00)


In [127]:
trange_llc.head()


file_id       proc_id io_cat   duration acc_pat            size  \
                                                 sum     min max         min   
trange                                                                         
5       1.450155e+20  2.195242e+20   96.0  66.775764       0   0         0.0   
450     6.706967e+20  1.015299e+21  432.0   5.333155       0   0  28713714.0   
668     1.255290e+21  1.900256e+21  819.0   4.466336       0   0  28713714.0   
344     6.253793e+20  9.466981e+20  408.0   4.180270       0   2  33560897.0   
233     5.075542e+20  7.683347e+20  331.0   3.813165       0   2  16783681.0   

                                 index  
               max          sum  count  
trange                                  
5              0.0          0.0   32.0  
450     50602738.0  146154226.0  567.0  
668     50602738.0  146154226.0  909.0  
344     33609917.0   67164349.0  535.0  
233     16816361.0   33593577.0  374.0

time: 50.5 ms (started: 2023-01-24 21:51:09 -08:00)


In [133]:
all_agg_persisted[all_agg_persisted['trange'].isin(trange_view_indices)].groupby(['trange', 'file_id', 'proc_id', 'io_cat'])

time: 158 ms (started: 2023-01-24 23:30:27 -08:00)


In [130]:
trange_llc_extended = all_agg_persisted[all_agg_persisted['trange'].isin(trange_view_indices)] \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration': [sum],
        'acc_pat': [min, max],
        'size': [min, max, sum],
        # 'index': ['count']
    }) \
    .persist()

trange_llc_extended


ValueError: Metadata inference failed in `_groupby_apply_funcs`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
KeyError("None of [Index(['acc_pat'], dtype='object')] are in the [columns]")

Traceback:
---------
  File "/g/g92/izzet/projects/vani-analysis-tool/venv-quartz/lib/python3.7/site-packages/dask/dataframe/utils.py", line 176, in raise_on_meta_error
    yield
  File "/g/g92/izzet/projects/vani-analysis-tool/venv-quartz/lib/python3.7/site-packages/dask/dataframe/core.py", line 5984, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "/g/g92/izzet/projects/vani-analysis-tool/venv-quartz/lib/python3.7/site-packages/dask/dataframe/groupby.py", line 920, in _groupby_apply_funcs
    r = func(grouped, *

time: 90.1 ms (started: 2023-01-24 23:30:14 -08:00)


In [140]:
all_agg_persisted[all_agg_persisted['trange'].isin(trange_view_indices)].compute().set_index(['trange', 'file_id', 'proc_id', 'io_cat'])

duration acc_pat      \
                                                            sum     min max   
trange file_id             proc_id             io_cat                         
3      901974072163427748  6860131432589814420 3       0.000003       0   0   
                           6860131432589879956 3       0.000003       0   0   
                           6860131432589945492 3       0.000002       0   0   
                           6860131432590011028 3       0.000003       0   0   
                           6860131432590076564 3       0.000002       0   0   
...                                                         ...     ...  ..   
669    4531734246508946226 6860131432660527764 3       0.000006       0   0   
                           6860131432663149204 3       0.000005       0   0   
                           6860131432665770644 3       0.000005       0   0   
                           6860131432668392084 3       0.000005       0   0   
                           6860131432671013524 3       0.000006       0   0   

                                                      size         index  
                                                       min max sum count  
trange file_id             proc_id             io_cat                     
3      901974072163427748  6860131432589814420 3         0   0   0     1  
                           6860131432589879956 3         0   0   0     1  
                           6860131432589945492 3         0   0   0     1  
                           6860131432590011028 3         0   0   0     1  
                           6860131432590076564 3         0   0   0     1  
...                                                    ...  ..  ..   ...  
669    4531734246508946226 6860131432660527764 3         0   0   0     1  
                           6860131432663149204 3         0   0   0     1  
                           6860131432665770644 3         0   0   0     1  
                           6860131432668392084 3         0   0   0     1  
                           6860131432671013524 3         0   0   0     1  

[6045 rows x 7 columns]

time: 90.2 ms (started: 2023-01-24 23:32:54 -08:00)
